# Melt Columns into Rows

In [1]:
import pandas as pd

In [2]:
# Read the Excel file
df = pd.read_excel('list of miracles.xlsx', sheet_name='References')

In [3]:
df.head()

,desc,ref1,ref2,ref3,ref4,ref5
0,The virgin birth of Jesus,Matthew 1:18-25,Luke 1:26-38,NaN,NaN,NaN
1,Jesus turns water into wine at Cana in Galilee,John 2:1-11,NaN,NaN,NaN,NaN
2,A royal official’s son is healed at Cana,John 4:46-54,NaN,NaN,NaN,NaN
3,Demoniac in synagogue healed in Capernaum,Mark 1:21-28,Luke 4:33-37,NaN,NaN,NaN
4,Healing of Peter’s mother-in-law,Matthew 8:14-15,Mark 1:29-31,Luke 4:38-39,NaN,NaN


In [4]:
# Create an order column to preserve original desc order
df['original_order'] = range(len(df))

In [5]:
# Melt the dataframe to convert ref columns to rows
melted_df = pd.melt(df, 
                    id_vars=['desc', 'original_order'], 
                    value_vars=['ref1', 'ref2', 'ref3', 'ref4', 'ref5'],
                    var_name='ref_type', 
                    value_name='ref_value')

In [6]:
# Remove rows where ref_value is NaN
melted_df = melted_df.dropna(subset=['ref_value'])

In [7]:
# Sort by original_order first, then by ref_type to maintain both original desc order
# and ref1, ref2, ref3, ref4, ref5 order within each desc
melted_df = melted_df.sort_values(['original_order', 'ref_type'])

In [8]:
# Create final dataframe with just desc and ref values
result_df = melted_df[['desc', 'ref_value']].reset_index(drop=True)

In [12]:
# Initialize new columns
result_df['book'] = None
result_df['chapter_start'] = None
result_df['chapter_end'] = None
result_df['verse_start'] = None
result_df['verse_end'] = None

# Split ref_value into components
for idx in result_df.index:
    ref = result_df.loc[idx, 'ref_value']
    
    if ref is not None:
        # Find the last space before a number to separate book from chapter:verse
        parts = ref.split()
        
        # Find where the numeric part starts
        book_parts = []
        numeric_part = None
        
        for i, part in enumerate(parts):
            # Check if this part starts with a number (could be chapter or chapter:verse)
            if part[0].isdigit():
                book_parts = parts[:i]
                numeric_part = ' '.join(parts[i:])
                break
        
        if book_parts and numeric_part:
            result_df.loc[idx, 'book'] = ' '.join(book_parts)
            
            # Now parse the numeric part
            if ':' in numeric_part:
                # Has verses: "1:9-12" or "1:9"
                chapter_part, verse_part = numeric_part.split(':', 1)
                
                # Handle chapter range before colon (unlikely but possible)
                if '-' in chapter_part:
                    ch_start, ch_end = chapter_part.split('-', 1)
                    result_df.loc[idx, 'chapter_start'] = int(ch_start)
                    result_df.loc[idx, 'chapter_end'] = int(ch_end)
                else:
                    result_df.loc[idx, 'chapter_start'] = int(chapter_part)
                    result_df.loc[idx, 'chapter_end'] = int(chapter_part)
                
                # Handle verse range
                if '-' in verse_part:
                    verse_start, verse_end = verse_part.split('-', 1)
                    result_df.loc[idx, 'verse_start'] = int(verse_start)
                    result_df.loc[idx, 'verse_end'] = int(verse_end)
                else:
                    # Single verse
                    result_df.loc[idx, 'verse_start'] = int(verse_part)
                    result_df.loc[idx, 'verse_end'] = int(verse_part)
                    
            else:
                # No verses, just chapter(s): "16" or "1-22"
                if '-' in numeric_part:
                    # Chapter range: "1-22"
                    ch_start, ch_end = numeric_part.split('-', 1)
                    result_df.loc[idx, 'chapter_start'] = int(ch_start)
                    result_df.loc[idx, 'chapter_end'] = int(ch_end)
                else:
                    # Single chapter: "16"
                    result_df.loc[idx, 'chapter_start'] = int(numeric_part)
                    result_df.loc[idx, 'chapter_end'] = int(numeric_part)

# Reset index after sorting
result_df = result_df.reset_index(drop=True)

                                                  desc              ref_value  \
0                            The virgin birth of Jesus        Matthew 1:18-25   
1                            The virgin birth of Jesus           Luke 1:26-38   
2       Jesus turns water into wine at Cana in Galilee            John 2:1-11   
3             A royal official’s son is healed at Cana           John 4:46-54   
4            Demoniac in synagogue healed in Capernaum           Mark 1:21-28   
..                                                 ...                    ...   
203             Shadow goes back 20 degrees on sundial        2 Kings 20:9-11   
204                      Uzziah afflicted with leprosy  2 Chronicles 26:16-21   
205             Three men delivered from fiery furnace         Daniel 3:19-27   
206                   Daniel delivered from lion’s den         Daniel 6:16-23   
207  Preservation of Jonah in the belly of fish for...           Jonah 2:1-10   

        book chapter_start 

In [24]:
result_df['book'] = None
result_df['chapter_start'] = None
result_df['chapter_end'] = None
result_df['verse_start'] = None
result_df['verse_end'] = None

# Split ref_value into components
for idx in result_df.index:
    ref = result_df.loc[idx, 'ref_value']
    
    if ref is not None:
        # Find the last occurrence of a pattern that looks like chapter:verse or just chapter
        # Look for patterns like "20:9-11" or "16" or "1-22"
        import re
        
        # Pattern to match chapter:verse or just chapter at the end
        # This matches: optional chapters followed by optional :verses
        pattern = r'^(.+?)\s+(\d+(?:-\d+)?(?::\d+(?:-\d+)?)?)$'
        match = re.match(pattern, ref.strip())
        
        if match:
            book_name = match.group(1)
            numeric_part = match.group(2)
            
            result_df.loc[idx, 'book'] = book_name
            
            # Now parse the numeric part
            if ':' in numeric_part:
                # Has verses: "20:9-11" or "1:9"
                chapter_part, verse_part = numeric_part.split(':', 1)
                
                # Handle chapter range before colon (unlikely but possible)
                if '-' in chapter_part:
                    ch_start, ch_end = chapter_part.split('-', 1)
                    result_df.loc[idx, 'chapter_start'] = int(ch_start)
                    result_df.loc[idx, 'chapter_end'] = int(ch_end)
                else:
                    result_df.loc[idx, 'chapter_start'] = int(chapter_part)
                    result_df.loc[idx, 'chapter_end'] = int(chapter_part)
                
                # Handle verse range
                if '-' in verse_part:
                    verse_start, verse_end = verse_part.split('-', 1)
                    result_df.loc[idx, 'verse_start'] = int(verse_start)
                    result_df.loc[idx, 'verse_end'] = int(verse_end)
                else:
                    # Single verse
                    result_df.loc[idx, 'verse_start'] = int(verse_part)
                    result_df.loc[idx, 'verse_end'] = int(verse_part)
                    
            else:
                # No verses, just chapter(s): "16" or "1-22"
                if '-' in numeric_part:
                    # Chapter range: "1-22"
                    ch_start, ch_end = numeric_part.split('-', 1)
                    result_df.loc[idx, 'chapter_start'] = int(ch_start)
                    result_df.loc[idx, 'chapter_end'] = int(ch_end)
                    result_df.loc[idx, 'verse_start'] = 1
                    result_df.loc[idx, 'verse_end'] = None
                else:
                    # Single chapter: "16"
                    result_df.loc[idx, 'chapter_start'] = int(numeric_part)
                    result_df.loc[idx, 'chapter_end'] = int(numeric_part)
                    result_df.loc[idx, 'verse_start'] = 1
                    result_df.loc[idx, 'verse_end'] = None                    

# Reset index after sorting
result_df = result_df.reset_index(drop=True)

In [25]:
result_df

,desc,ref_value,book,chapter_start,chapter_end,verse_start,verse_end
0,The virgin birth of Jesus,Matthew 1:18-25,Matthew,1,1,18,25
1,The virgin birth of Jesus,Luke 1:26-38,Luke,1,1,26,38
2,Jesus turns water into wine at Cana in Galilee,John 2:1-11,John,2,2,1,11
3,A royal official’s son is healed at Cana,John 4:46-54,John,4,4,46,54
4,Demoniac in synagogue healed in Capernaum,Mark 1:21-28,Mark,1,1,21,28
...,...,...,...,...,...,...,...
203,Shadow goes back 20 degrees on sundial,2 Kings 20:9-11,2 Kings,20,20,9,11
204,Uzziah afflicted with leprosy,2 Chronicles 26:16-21,2 Chronicles,26,26,16,21
205,Three men delivered from fiery furnace,Daniel 3:19-27,Daniel,3,3,19,27
206,Daniel delivered from lion’s den,Daniel 6:16-23,Daniel,6,6,16,23


In [28]:
result_df[result_df.book=='Revelation']

,desc,ref_value,book,chapter_start,chapter_end,verse_start,verse_end
113,John’s vision on Patmos,Revelation 1-22,Revelation,1,22,1,None


In [29]:
result_df.to_csv('miracle_references.csv')